In [1]:
import pandas as pd
import numpy as np

# Competition을 위한 데이터 처리 예제

- mem_data.csv 파일의 GENDER 열과 MEM_ID열은 모델링에 사용됨에 따라 수정하시면 안됩니다.

In [86]:
mem_data = pd.read_csv('data/mem_data.csv')
mem_tr = pd.read_csv('data/mem_transaction.csv')
s_info = pd.read_csv('data/store_info.csv')

In [87]:
mem_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10764 entries, 0 to 10763
Data columns (total 15 columns):
MEM_ID         10764 non-null int64
M_STORE_ID     10764 non-null int64
GENDER         10764 non-null object
BIRTH_DT       5997 non-null object
BIRTH_SL       10764 non-null object
ZIP_CD         10764 non-null object
RGST_DT        10764 non-null object
VISIT_CNT      10764 non-null float64
SALES_AMT      10764 non-null float64
LAST_VST_DT    10764 non-null object
USABLE_PNT     10764 non-null float64
USED_PNT       10764 non-null float64
ACC_PNT        10764 non-null float64
USABLE_INIT    10764 non-null float64
SMS            10764 non-null object
dtypes: float64(6), int64(2), object(7)
memory usage: 1.2+ MB


In [88]:
mem_data.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS
0,1134945,1084,M,NaN,S,-,2006-02-04 11:51:08.233000000,8.0,337000.0,2007-01-19 21:46:12.827000000,6740.0,0.0,6740.0,3202.0,Y
1,38458,539,F,1973-07-11,S,420-721,2006-02-04 11:51:08.233000000,22.0,1438500.0,2007-10-18 19:57:45.503000000,1732.0,12000.0,13732.0,17762.0,Y
2,7009,1113,F,1973-04-06,S,-,2006-02-04 11:51:08.233000000,8.0,309300.0,2007-05-30 15:21:10.863000000,6372.0,0.0,6372.0,1856.0,Y
3,91791,2273,UNKNOWN,1968-09-05,S,487-820,2006-02-04 11:51:08.233000000,11.0,170400.0,2007-10-16 15:55:52.483000000,3408.0,0.0,3408.0,236.0,Y
4,1374842,1300,M,NaN,S,-,2006-02-04 11:51:08.233000000,3.0,110300.0,2007-03-25 12:49:30.170000000,2206.0,0.0,2206.0,1522.0,Y


In [89]:
mem_data.BIRTH_SL.value_counts()

S    8973
L    1791
Name: BIRTH_SL, dtype: int64

In [90]:
mem_tr.head()

,STORE_ID,SELL_DT,MEMP_STY,MEM_ID,MEMP_DT,SELL_AMT,MEMP_TP
0,125,20060227,O,1225434,2006-02-27 13:57:44.750000000,15500,A
1,125,20060303,O,1181938,2006-03-03 15:43:56.987000000,22600,A
2,125,20060322,O,6093,2006-03-22 18:08:45.563000000,11000,A
3,125,20060403,O,1398202,2006-04-03 13:36:27.040000000,14400,A
4,125,20060418,O,1225434,2006-04-18 16:04:39.583000000,6600,A


In [91]:
s_info.head()

,STORE_ID,STORE_REGION
0,128,2
1,129,2
2,130,11
3,133,16
4,136,13


In [92]:
mem_data.GENDER.value_counts()

F          3767
M          3767
UNKNOWN    3230
Name: GENDER, dtype: int64

In [93]:
mem_data.SMS.value_counts()

Y    10575
N      189
Name: SMS, dtype: int64

In [94]:
mem_data.BIRTH_DT.isnull().sum()

4767

## **[mem_data 처리]**

**[변수 처리 1]** SMS 수신동의 정수 처리하기

In [95]:
mem_data.SMS.value_counts()

Y    10575
N      189
Name: SMS, dtype: int64

In [96]:
mem_data.SMS = (mem_data.SMS=='Y').astype(int)
mem_data.SMS.value_counts()

1    10575
0      189
Name: SMS, dtype: int64

**[변수 처리 2]** 양/음력(BIRTH_SL) 정수 처리하기 

In [97]:
mem_data.BIRTH_SL = (mem_data.BIRTH_SL=='Y').astype(int)
mem_data.BIRTH_SL.value_counts()

0    10764
Name: BIRTH_SL, dtype: int64

**[변수 처리 3]** 구매 합계(SALES_AMT) 로그 처리 하기
- 참고 : 로그처리에는 음수가 들어갈 수 없음 / 0이 있을 경우 1을 더하고 처리

In [98]:
f = mem_data.SALES_AMT.where(mem_data.SALES_AMT>=0, other=0) # 음수처리
f = np.log(f+1)
mem_data.SALES_AMT = f

**[변수 처리 4]** 로그 처리가 필요하다고 생각되는 변수를 로그처리 하기

In [99]:
f = mem_data.USABLE_PNT.where(mem_data.SALES_AMT>=0, other=0) # 음수처리
f = np.log(f+1)
mem_data.USABLE_PNT = f

In [100]:
f = mem_data.USED_PNT.where(mem_data.SALES_AMT>=0, other=0) # 음수처리
f = np.log(f+1)
mem_data.USED_PNT = f

In [101]:
f = mem_data.ACC_PNT.where(mem_data.SALES_AMT>=0, other=0) # 음수처리
f = np.log(f+1)
mem_data.ACC_PNT = f

In [102]:
f = mem_data.USABLE_INIT.where(mem_data.SALES_AMT>=0, other=0) # 음수처리
f = np.log(f+1)
mem_data.USABLE_INIT = f

In [103]:
mem_data.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS
0,1134945,1084,M,NaN,0,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,8.815964,0.000000,8.815964,8.071843,1
1,38458,539,F,1973-07-11,0,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,7.457609,9.392745,9.527557,9.784873,1
2,7009,1113,F,1973-04-06,0,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,8.759826,0.000000,8.759826,7.526718,1
3,91791,2273,UNKNOWN,1968-09-05,0,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,8.134174,0.000000,8.134174,5.468060,1
4,1374842,1300,M,NaN,0,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,7.699389,0.000000,7.699389,7.328437,1


**[변수 생성 1]** 최근 방문 일자(LAST_VST_DT)로부터 경과일 구하기

In [104]:
f = pd.to_datetime(mem_data.LAST_VST_DT) # 날짜가 Object 이므로 숫자 처리
f = (pd.to_datetime('2007-12-13') - f).dt.days #방문이 얼마나 오래되었는지 ,time 등으로 알고 싶은 것으로 바꿀 수 있다.
mem_data['E_DAY'] = f
mem_data.E_DAY.head()

0    327
1     55
2    196
3     57
4    262
Name: E_DAY, dtype: int64

In [105]:
f = pd.to_datetime(mem_data.LAST_VST_DT)

In [106]:
f = (pd.to_datetime('2007-12-13') - f)

In [107]:
f[f.isna()]

Series([], Name: LAST_VST_DT, dtype: timedelta64[ns])

In [108]:
mem_data.LAST_VST_DT[1612]

'2006-06-02 11:31:20.640000000'

In [109]:
f.isna().sum()

0

**[변수 생성 2]** 등록일(RGST_DT)로부터 경과일 구하기

In [110]:
f = pd.to_datetime(mem_data.RGST_DT) # 날짜가 Object 이므로 숫자 처리
f = (pd.to_datetime('2007-12-13') - f).dt.days #방문이 얼마나 오래되었는지 ,time 등으로 알고 싶은 것으로 바꿀 수 있다.
mem_data['RG_DAY'] = f
mem_data.RG_DAY.head(30)

0     676
1     676
2     676
3     676
4     676
5     676
6     675
7     676
8     676
9     676
10    676
11    676
12    667
13    676
14    676
15    676
16    676
17    676
18    676
19    676
20    676
21    676
22    676
23    676
24    676
25    676
26    676
27    676
28    676
29    676
Name: RG_DAY, dtype: int64

In [111]:
mem_data.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,E_DAY,RG_DAY
0,1134945,1084,M,NaN,0,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,8.815964,0.000000,8.815964,8.071843,1,327,676
1,38458,539,F,1973-07-11,0,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,7.457609,9.392745,9.527557,9.784873,1,55,676
2,7009,1113,F,1973-04-06,0,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,8.759826,0.000000,8.759826,7.526718,1,196,676
3,91791,2273,UNKNOWN,1968-09-05,0,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,8.134174,0.000000,8.134174,5.468060,1,57,676
4,1374842,1300,M,NaN,0,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,7.699389,0.000000,7.699389,7.328437,1,262,676


**[변수 생성 3]** 우편번호(ZIP_CD)에서 광역행정구역 데이터 가져오기
- 참고: 6자리 우편번호의 구성 : https://ko.wikipedia.org/wiki/대한민국의_우편번호#6자리_우편번호_(1988~2015)

In [112]:
mem_data.ZIP_CD.head()

0          -
1    420-721
2          -
3    487-820
4          -
Name: ZIP_CD, dtype: object

In [134]:
mem_data.ZIP_CD[1][2]

'0'

In [114]:
# 리스트를 구할때 첫번째 글자만 가져올때
f = [x[0] for x in mem_data.ZIP_CD]

In [115]:
f = [x[0] for x in mem_data.ZIP_CD]
mem_data['R_REGION'] = f
mem_data.R_REGION = mem_data.R_REGION.where(mem_data.R_REGION != '-', other=0).astype(int)
mem_data.R_REGION.head()

0    0
1    4
2    0
3    4
4    0
Name: R_REGION, dtype: int32

In [116]:
mem_data.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,E_DAY,RG_DAY,R_REGION
0,1134945,1084,M,NaN,0,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,8.815964,0.000000,8.815964,8.071843,1,327,676,0
1,38458,539,F,1973-07-11,0,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,7.457609,9.392745,9.527557,9.784873,1,55,676,4
2,7009,1113,F,1973-04-06,0,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,8.759826,0.000000,8.759826,7.526718,1,196,676,0
3,91791,2273,UNKNOWN,1968-09-05,0,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,8.134174,0.000000,8.134174,5.468060,1,57,676,4
4,1374842,1300,M,NaN,0,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,7.699389,0.000000,7.699389,7.328437,1,262,676,0


**[변수 생성 4]** 우편번호(ZIP_CD)에서 광역행정구역과 구를 합쳐서 데이터 가져오기

## **[다른 데이터와 연동하여 변수 생성]**

**[변수 생성 1]** 평균 구매액 구하기

In [138]:
f = mem_tr.groupby('MEM_ID')['SELL_AMT'].agg({'TOTAL_AMT':'mean'}).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

C:\ProgramData\Anaconda3\envs\[tf]\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


**[변수 생성 2]** 포인트 적립 횟수 구하기

In [139]:
f = mem_tr[mem_tr.MEMP_TP=='A'].groupby('MEM_ID')['SELL_AMT'].agg({'POINT_N':'size'}).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

C:\ProgramData\Anaconda3\envs\[tf]\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


**[변수 생성 3]** 요일 구매 패턴 구하기 : 주중형 / 주말형

In [140]:
mem_data.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,E_DAY,RG_DAY,R_REGION,TOTAL_AMT,POINT_N
0,1134945,1084,M,NaN,0,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,8.815964,0.000000,8.815964,8.071843,1,327,676,0,16262.500000,8
1,38458,539,F,1973-07-11,0,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,7.457609,9.392745,9.527557,9.784873,1,55,676,4,25018.181818,22
2,7009,1113,F,1973-04-06,0,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,8.759826,0.000000,8.759826,7.526718,1,196,676,0,22025.000000,8
3,91791,2273,UNKNOWN,1968-09-05,0,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,8.134174,0.000000,8.134174,5.468060,1,57,676,4,14418.181818,11
4,1374842,1300,M,NaN,0,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,7.699389,0.000000,7.699389,7.328437,1,262,676,0,11400.000000,3


**[변수 생성 4]** 시간대별 표인트 적립 건수 구하기 : Morning(09-12) / Afternoon(13-17) / Evening(18-20)

**[변수 처리]** 생성한 변수중 큰 값들 로그 처리 

In [141]:
f1 = mem_data.TOTAL_AMT.where(mem_data.SALES_AMT>=0, other=0) # 음수처리
f1 = np.log(f1+1)
mem_data.TOTAL_AMT = f1

In [142]:
mem_data

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,E_DAY,RG_DAY,R_REGION,TOTAL_AMT,POINT_N
0,1134945,1084,M,NaN,0,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,8.815964,0.000000,8.815964,8.071843,1,327,676,0,9.696679,8
1,38458,539,F,1973-07-11,0,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,7.457609,9.392745,9.527557,9.784873,1,55,676,4,10.127398,22
2,7009,1113,F,1973-04-06,0,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,8.759826,0.000000,8.759826,7.526718,1,196,676,0,9.999979,8
3,91791,2273,UNKNOWN,1968-09-05,0,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,8.134174,0.000000,8.134174,5.468060,1,57,676,4,9.576315,11
4,1374842,1300,M,NaN,0,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,7.699389,0.000000,7.699389,7.328437,1,262,676,0,9.341456,3
5,719600,130,UNKNOWN,1977-01-23,0,-,2006-02-04 11:51:08.233000000,3.0,10.793455,2007-07-19 22:04:01.567000000,6.882437,0.000000,6.882437,4.890349,1,146,676,0,9.549262,3
6,1383660,1024,M,NaN,0,-,2006-02-05 16:56:58,2.0,10.419331,2006-09-09 14:55:21.687000000,6.508769,0.000000,6.508769,4.709530,1,459,675,0,9.546884,2
7,911717,10792,M,NaN,0,-,2006-02-04 11:51:08.233000000,18.0,12.334470,2007-10-21 17:16:03.257000000,8.422663,0.000000,8.422663,6.736967,1,52,676,0,9.239457,18
8,172362,799,UNKNOWN,NaN,0,-,2006-02-04 11:51:08.233000000,2.0,10.968216,2006-07-31 15:13:01.703000000,7.112327,0.000000,7.112327,6.240276,1,499,676,0,9.692828,2
9,272744,997,UNKNOWN,1978-11-04,0,-,2006-02-04 11:51:08.233000000,42.0,14.279308,2007-10-31 14:34:27.783000000,9.223750,10.043293,10.408345,8.993800,1,42,676,0,10.363928,36


**[날짜 데이터 처리]** 날짜표기 데이터들 날짜로 표시 와 특정 컬럼 뽑기

In [154]:
import datetime

In [160]:
datetime(mem_data.BIRTH_DT[1][0:4], mem_data.BIRTH_DT[1][6:8], mem_data.BIRTH_DT[1][9:11])

TypeError: 'module' object is not callable

In [178]:
datetime.datetime.strptime(mem_data["BIRTH_DT"][1], "%Y-%m-%d")

datetime.datetime(1973, 7, 11, 0, 0)

In [170]:
mem_data["BIRTH_DT"].str[5:7][:5]

0    NaN
1     07
2     04
3     09
4    NaN
Name: BIRTH_DT, dtype: object

## **[최종 결과 저장]**

- mem_data에 최종 결과가 모일 수 있도록 준비
- 1.예측에 사용하지 않을 열들 drop
- 2.csv형태로 저장
- **주의사항1 : MEM_ID열은 삭제하지 마세요. **
- MEM_ID를 변수로 사용하기를 원하면 다른 열을 새로 생성해서 복사해서 사용하세요.
- 주의사항2 : GENDER열을 제외하고 문자열이 포함된 열이 데이터에 포함되지 않도록 한다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10764 entries, 0 to 10763
Data columns (total 19 columns):
MEM_ID         10764 non-null int64
M_STORE_ID     10764 non-null int64
GENDER         10764 non-null object
BIRTH_DT       5997 non-null object
BIRTH_SL       10764 non-null int32
ZIP_CD         10764 non-null object
RGST_DT        10764 non-null object
VISIT_CNT      10764 non-null float64
SALES_AMT      10764 non-null float64
LAST_VST_DT    10764 non-null object
USABLE_PNT     10764 non-null float64
USED_PNT       10764 non-null float64
ACC_PNT        10764 non-null float64
USABLE_INIT    10764 non-null float64
SMS            10764 non-null int32
E_DAY          10764 non-null int64
R_REGION       10764 non-null int32
TOTAL_AMT      10764 non-null float64
POINT_N        10764 non-null int64
dtypes: float64(7), int32(3), int64(4), object(5)
memory usage: 1.5+ MB


In [25]:
d_col = ['BIRTH_SL','M_STORE_ID','BIRTH_DT','ZIP_CD','RGST_DT','LAST_VST_DT']
mem_data = mem_data.drop(d_col, axis=1)
mem_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10764 entries, 0 to 10763
Data columns (total 13 columns):
MEM_ID         10764 non-null int64
GENDER         10764 non-null object
VISIT_CNT      10764 non-null float64
SALES_AMT      10764 non-null float64
USABLE_PNT     10764 non-null float64
USED_PNT       10764 non-null float64
ACC_PNT        10764 non-null float64
USABLE_INIT    10764 non-null float64
SMS            10764 non-null int32
E_DAY          10764 non-null int64
R_REGION       10764 non-null int32
TOTAL_AMT      10764 non-null float64
POINT_N        10764 non-null int64
dtypes: float64(7), int32(2), int64(3), object(1)
memory usage: 1.1+ MB


In [26]:
mem_data.head()

,MEM_ID,GENDER,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,E_DAY,R_REGION,TOTAL_AMT,POINT_N
0,1134945,M,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,0,16262.500000,8
1,38458,F,22.0,14.179112,1732.0,12000.0,13732.0,17762.0,1,55,4,25018.181818,22
2,7009,F,8.0,12.642070,6372.0,0.0,6372.0,1856.0,1,196,0,22025.000000,8
3,91791,UNKNOWN,11.0,12.045910,3408.0,0.0,3408.0,236.0,1,57,4,14418.181818,11
4,1374842,M,3.0,11.610968,2206.0,0.0,2206.0,1522.0,1,262,0,11400.000000,3


In [27]:
mem_data.to_csv('final_data.csv', index=False)

In [28]:
final_data = pd.read_csv('final_data.csv')

In [29]:
final_data.dtypes

MEM_ID           int64
GENDER          object
VISIT_CNT      float64
SALES_AMT      float64
USABLE_PNT     float64
USED_PNT       float64
ACC_PNT        float64
USABLE_INIT    float64
SMS              int64
E_DAY            int64
R_REGION         int64
TOTAL_AMT      float64
POINT_N          int64
dtype: object